# DENUE Exploratory Data Analysis notebook

[DENUE](http://www.beta.inegi.org.mx/app/mapa/denue/) means Directorio Estadístico Nacional de Unidades Economicas. It is maintained by [INEGI](http://www.inegi.org.mx/), the Mexican Institute of Statistics and Geography.

In this notebook we start exploring the data as part of our project to map the creative industries in Mexico

## About the data

This [methodological note](http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825097240) describes the data (in Spanish).

DENUE contains information about ~5 million active businesses in Mexico. This includes:

* Identification
* Location
* Sector
* Size

DENUE’s first edition was published in 2010, with data collected in 2009. Individual entrepreneurs and organisations are legally required to register in DENUE and update their information. Other parts of the public sector are also required to provide INEGI with data to maintain DENUE. 

**Data updates**

* Information about larger businesses (>20 employees / $20m in turnover or activities in more than one State) as well as those in some sectors are updated yearly with administrative data and business surveys.

* Information about micro/small/medium businesses is partially updated through administrative registers.

* Users update their information online continuously, and this information is quality assured by INEGI.

* All data in DENUE are updated every 5 years through information produced by the economic census. The latest complete update of DENUE was in 2016, using data from the 2014 economic census.

### Conceptual framework and coverage

**Unit of analysis**

DENUE contains information about establishments (local units) and enterprises. 

**Sector**

DENUE contains information about all industries with the exception of agriculture, farming and forestry, transport, political organisations, domestic workers and extraterritorial units.

Industrial activities are classified according to 2013 NAICS. The first edition of DENUE using 2013 NAICs was in 2015 (*this means we can probably use definitions of the creative industries implemented by Nesta in its analysis of the creative industries in North America*)

**Period**

The period covered by business data depends on the year of incorporation.
  * For businesses incorporated before 2014, DENUE data refers to 2014
  * For businesses incorporated Jan 2015- Oct 2016 - DENUE data refers to 2015
  * For business incorporated Mar 2017-Nov 2017 - Data refers to 2016

**Geography**

DENUE includes economic units in locations with 2500+ inhabitants as well as ‘economic locations’ such as industrial parks. It doesn’t contain economic units in rural areas./

Data are available at the following levels of resolution:
* AGEE (States) - 2 digit based on State name. Administrative boundary
* AGEM (towns / cities) - 3 digits.
* AGEB (micro-geography):
  * Urban: 1-50 squares
  * Rural: ~11m Ha.

Longitude and Latitude data are also available. 

### Variables

* Nombre: Name. Some organisations have generic names.
* Denominación o razón social y tipo de sociedad: Type of organisation
* Código: industrial code based on NAICs (organisations are allocated to the sector which generates most turnover for the company)
* Personal: 8 sizebands (0-251+)
* Tipo de Unidad económica: 
  * Whether the organisation is a subsidiary, and the type of establishment.
* Datos de ubicación: Location (address)
* Area geográfica: (see above)
* Contact details:
  * Telephone
  * Web address
  * Email



## Some processing activities

* Load data
* Merge with creative NAICs codes
* Produce some descriptive analysis: creative sectors by location (totals, specialisation)
* Analyse access to contact details




## Preamble

This includes:

* Package imports
* Paths
* Functions and classes used below



In [218]:
%matplotlib inline

#Key imports are loaded from my profile (see standard_imports.py in src folder).

#Other imports 

#Zipfile to work with zips.
import zipfile
import csv

#Paths

#Paths
top = os.path.dirname(os.getcwd())

#External data
ext_data = os.path.join(top,'data/external')

#Metadata
meta_data = os.path.join(top,'data/metadata')

#Interim data
int_data = os.path.join(top,'data/interim')

#Processed data
proc_data = os.path.join(top,'data/processed')

#Figures
fig_path = os.path.join(top,'reports/final_figures')

#Tables
tab_path = os.path.join(top,'reports/tables')

#Models
mod_path = os.path.join(top,'models')

#Visualisations
viz_path = os.path.join(top,'src/visualization')

#Get date for saving files
today = datetime.datetime.today()

today_str = "_".join([str(x) for x in [today.day,today.month,today.year]])

## 1. Load data

Here we will process all relevant DENUE files. There are quite a few of them and they are big. To deal with this I will write a class that loads them one at a time, unzips them, extracts companies in creative sectors (based on a lookup table based on NAICS codes) and generates some sector / area reports to normalise findings.


### a. Load metadata

Lookup between NAICs and creative industries codes based on Nesta 2016 (see Table A10 in /references folder)

Some of the codes in that table are too coarse (ie advertising also includes management consultancy). We have created a more refined classification that removes some of the non-creative subcodes, but we will generally work with Nesta's classification for consistency. It would be easy to replicate the analysis focusing on the refined categories


In [83]:
#Load a file with NAICs codes and info about whether they are creative or not
naics_creative_lookup = pd.read_csv(meta_data+'/naics_sics_creative_lookup.csv')

#Flag creative codes
naics_creative_lookup['has_creative'] = [1 if x==1 or y==1 else 0 for x,y in zip(
    naics_creative_lookup['creative_nesta_2016'],
    naics_creative_lookup['creative_refined'])]

#Focus on relevant codes
naics_creative_lookup_short = naics_creative_lookup.loc[
    naics_creative_lookup.has_creative==1,:].reset_index(drop=True)

#Turn codes into ints
naics_creative_lookup_short['code_int'] =[int(x) for x in naics_creative_lookup_short.code_es]

### b. Load one file for exploration

In [30]:
#This creates the file. We will load one that has creative sectors in it
zf = zipfile.ZipFile(ext_data+'/sector_data/Actividad_económica.51_Información_en_medios_masivos.csv.zip')

#Extract the names
zf_names = zf.infolist()

print(zf_names)

[<ZipInfo filename='denue_00_51_csv/conjunto_de_datos/' external_attr=0x10>, <ZipInfo filename='denue_00_51_csv/conjunto_de_datos/denue_inegi_51_.csv' compress_type=deflate external_attr=0x20 file_size=8850882 compress_size=1894091>, <ZipInfo filename='denue_00_51_csv/diccionario_de_datos/' external_attr=0x10>, <ZipInfo filename='denue_00_51_csv/diccionario_de_datos/denue_diccionario_de_datos.csv' compress_type=deflate external_attr=0x20 file_size=9807 compress_size=3354>, <ZipInfo filename='denue_00_51_csv/metadatos/' external_attr=0x10>, <ZipInfo filename='denue_00_51_csv/metadatos/metadatos_denue.txt' compress_type=deflate external_attr=0x20 file_size=1987 compress_size=1055>]


#### Load the data dictionary

In [105]:
#Denue data dictionary

#This gets the filename for the data dictionary
data_dict_filename = zf_names[3].filename

#And we read it
denue_dictionary = pd.read_csv(zf.open(data_dict_filename))

#We save it so we can have a look at it, and maybe create cleaner names and variables
denue_dictionary.to_csv(meta_data+'/{date}_denue_data_dictionary.csv'.format(date=today_str))


#### Load the data

In [31]:
#We are interested in the 'denue_inegi_51_.csv' file, index=1
denue_data = zf_names[1]

#We get its name (which we will use to open the data)
denue_file_name = denue_data.filename

In [32]:
#Pandas can read this easily
denue_51 = pd.read_csv(zf.open(zf_file_name))

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
#Here it is
denue_51.head()

,id,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,...,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta
0,17294,"ALESTRA, S. DE R.L. DE C.V.","ALESTRA, S. DE R.L. DE C.V.",517110,Operadores de servicios de telecomunicaciones ...,0 a 5 personas,AVENIDA,UNIVERSIDAD,AVENIDA,AGUASCALIENTES NORTE,...,Aguascalientes,2259,19,4.499228e+09,MPAVON@ALESTRA.COM.MX,WWW.ALESTRA.COM.MX,Fijo,21.916712,-102.313562,MARZO 2011
1,42915,ALMACÉN DISH,"COMERCIALIZADORA DE FRECUENCIAS SATELITALES, S...",517910,Otros servicios de telecomunicaciones,51 a 100 personas,BOULEVARD,LICENCIADO MIGUEL DE LA MADRID,CALLE,LOS NARANJOS,...,Corral de Barrancos,043A,31,4.499943e+09,NaN,WWW.DISH.COM.MX,Fijo,21.949974,-102.332994,DICIEMBRE 2014
2,6506350,ALTARIA AGUASCALIENTES,"GRUPO IUSACELL, S.A. DE C.V.",517210,Operadores de servicios de telecomunicaciones ...,6 a 10 personas,CALLE,NINGUNO,NINGUNO,NINGUNO,...,Aguascalientes,1320,66,NaN,NaN,WWW.ATT.COM.MX,Fijo,21.925536,-102.289215,ENERO 2016
3,15779,ANIMACTIVA,NaN,511210,Edición de software y edición de software inte...,11 a 30 personas,CIRCUITO,HORTALIZAS,CALLE,FELIPE SERRANO,...,Aguascalientes,3098,7,4.491399e+09,cesar@animactiva.mx,www.animactiva.mx,Fijo,21.923799,-102.268272,JULIO 2010
4,20508,ARCHIVO GENERAL MUNICIPAL,PRESIDENCIA MUNICIPAL,519122,Bibliotecas y archivos del sector público,11 a 30 personas,CALLE,JUAN DE MONTORO,CALLE,JOSÉ MARÍA MORELOS Y PAVÓN,...,Aguascalientes,0638,6,4.499159e+09,MUNICIPAL@AGS.GOB.MX,WWW.AGS.GOB.MX/ARCHIVO,Fijo,21.881122,-102.293654,JULIO 2010


In [36]:
#It contains 19K obsrrvations and 41 variables
denue_51.shape

(19919, 41)

In [40]:
#These are the top 10 sectors
denue_51.nombre_act.value_counts()[:10]

Otros servicios de telecomunicaciones                                                 4518
Bibliotecas y archivos del sector público                                             4490
Operadores de servicios de telecomunicaciones alámbricas                              4082
Transmisión de programas de radio                                                     1241
Operadores de servicios de telecomunicaciones inalámbricas                             968
Exhibición de películas y otros materiales audiovisuales                               915
Edición de periódicos integrada con la impresión                                       727
Transmisión de programas de televisión                                                 396
Procesamiento electrónico de información, hospedaje y otros servicios relacionados     370
Edición de revistas y otras publicaciones periódicas                                   246
Name: nombre_act, dtype: int64

In [43]:
#What do the sector codes look like? Can we find them in the NAICs lookups

list(set(denue_51.codigo_act))[:5]

[512130, 511111, 511112, 515210, 511121]

They have 7 digits

In [50]:
#Check lookup. We do this by dropping duplicate codes in the data
code_name_lu = denue_51[['codigo_act','nombre_act']].drop_duplicates('codigo_act').reset_index(drop=True)

#We have compared the codes in the data with the codes in the NAICs file and they match.
#We will work with the NAICs file, enriched with the codes from Nesta (2016) report

### Merge DENUE and lookup

In [92]:
#Let's match this with the metadata to see what we get
denue_51_creative = pd.merge(denue_51,naics_creative_lookup_short,left_on='codigo_act',
                            right_on='code_int',how='left')

#Libraries and museums
denue_51_creative.loc[denue_51_creative.creative_sector=='software',:].head()

,id,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,...,fecha_alta,code_en,sector_spanish,code_es,sector_english,creative_nesta_2016,creative_refined,creative_sector,has_creative,code_int
3,15779,ANIMACTIVA,NaN,511210,Edición de software y edición de software inte...,11 a 30 personas,CIRCUITO,HORTALIZAS,CALLE,FELIPE SERRANO,...,JULIO 2010,511210,Edición de software y edición de software inte...,511210,"Software publishing, and integrated software p...",1.0,1.0,software,1.0,511210.0
59,39380,DIVERZA,"DIVERZA INFORMACIÓN Y ANÁLISIS, S.A.P.I. DE C....",518210,"Procesamiento electrónico de información, hosp...",6 a 10 personas,AVENIDA,AGUASCALIENTES NORTE,PROLONGACIÓN,GENERAL IGNACIO ZARAGOZA,...,DICIEMBRE 2014,518210,"Procesamiento electrónico de información, hosp...",518210,"Electronic data processing, hosting, and other...",1.0,1.0,software,1.0,518210.0
70,39206,FACTURACIÓN DIGITAL,"FACTURACIÓN DIGITAL, S.A. DE C.V.",518210,"Procesamiento electrónico de información, hosp...",6 a 10 personas,CALLE,PICO DE ORIZABA,CALLE,CORDILLERA DE LOS ALPES ORIENTE,...,DICIEMBRE 2014,518210,"Procesamiento electrónico de información, hosp...",518210,"Electronic data processing, hosting, and other...",1.0,1.0,software,1.0,518210.0
71,35838,FACTURACIÓN ELÉCTRONICA SIN NOMBRE,NaN,518210,"Procesamiento electrónico de información, hosp...",0 a 5 personas,CALLE,FRAY BARTOLOMÉ DE LAS CASAS,CALLE,ARQUITECTO J. REFUGIO REYES,...,DICIEMBRE 2014,518210,"Procesamiento electrónico de información, hosp...",518210,"Electronic data processing, hosting, and other...",1.0,1.0,software,1.0,518210.0
72,6765767,FACTURANET,FACTURANET SA DE CV,518210,"Procesamiento electrónico de información, hosp...",11 a 30 personas,AVENIDA,CONVENCIÓN DE 1914 NORTE,CALLE,20 DE NOVIEMBRE,...,OCTUBRE 2016,518210,"Procesamiento electrónico de información, hosp...",518210,"Electronic data processing, hosting, and other...",1.0,1.0,software,1.0,518210.0


In [93]:
#Creative sectors in the data
denue_51_creative.creative_sector.value_counts()

libraries_museums        4718
film_radio_tv            3066
publishing               1736
software                  449
music_performing_arts     158
Name: creative_sector, dtype: int64

In [97]:
#Sizes
pd.crosstab(denue_51_creative.per_ocu,denue_51_creative.creative_sector)

creative_sector,film_radio_tv,libraries_museums,music_performing_arts,publishing,software
per_ocu,,,,,
0 a 5 personas,1226,3975,117,910,236
101 a 250 personas,66,4,0,83,22
11 a 30 personas,667,259,16,277,67
251 y más personas,25,0,3,34,14
31 a 50 personas,350,33,5,79,23
51 a 100 personas,252,0,2,97,24
6 a 10 personas,480,447,15,256,63


In [111]:
#States
pd.crosstab(denue_51_creative.entidad,denue_51_creative.creative_sector)

creative_sector,film_radio_tv,libraries_museums,music_performing_arts,publishing,software
entidad,,,,,
AGUASCALIENTES,23,32,1,21,11
BAJA CALIFORNIA,110,73,2,38,8
BAJA CALIFORNIA SUR,32,40,1,18,4
CAMPECHE,40,38,0,27,4
CHIAPAS,134,196,6,39,6
CHIHUAHUA,115,144,4,42,20
COAHUILA DE ZARAGOZA,116,119,1,47,10
COLIMA,31,43,2,14,3
DISTRITO FEDERAL,439,295,54,361,109


In [121]:
#States
pd.crosstab(denue_51_creative.municipio,denue_51_creative.creative_sector)[:20]

creative_sector,film_radio_tv,libraries_museums,music_performing_arts,publishing,software
municipio,,,,,
Abalá,0,1,0,0,0
Abasolo,0,5,0,0,0
Abejones,0,1,0,0,0
Acacoyagua,0,2,0,0,0
Acajete,0,5,0,0,0
Acala,0,1,0,0,0
Acambay de Ruíz Castañeda,0,2,0,0,0
Acanceh,0,1,0,0,0
Acapetahua,0,2,0,0,0


## c. Load all data

**Plan**

I will write a class that takes a file name, reads it, extracts all creative businesses and also the distribution of companies in all other areas, as well as their size distribution (which we can use to normalise the data)

This involves:

1. Creating a list of all filenames I want to load
2. Writing a class with: 
 * A read method to read the CSV and rename the variables to something tidier
 * A get_creative method that merges the file with naics_creative lookup and returns all matches
 * An area_stats method that generates company counts by state and municipio
3. Implementing the class while I go for a run :-)


In [195]:
class ReadProcessDenue():
    
    '''
    DenueProcess is a class that takes a filename for a zip file and:
   
    -Extracts the csv content (read method)
    -Cleans variable names (process method)
    -Merges with the NAICS Lookup (get_creative method)
    -Generates area counts for all sectors (area_counts method).
    
    It stores the outputs in a creative_businesses attribute, a state_counts attribute and a local_counts 
    attribute. 

    '''
    
    def __init__(self,filepath):
        '''
        Initialise the class with a filepath
        
        '''
        
        self.filepath = filepath
        
    def read_file(self):
        '''
        Read the csv
        
        '''
        #Get the filepath
        filepath = self.filepath
        
        #Read the zipfile
        zf = zipfile.ZipFile(filepath)
        
        #Get the name of the datafile
        data_file_name = zf.infolist()[1].filename
        
        #Read the file
        denue_data = pd.read_csv(zf.open(data_file_name))
        
        #Assign the denue data to the denue_data attribute
        self.denue_data = denue_data

    
    def get_creative(self):
        '''
        Extracts creative companies
        
        '''
        #Read all denue data
        all_denue = self.denue_data
        
        #Merge with the creative lookup
        creative_denue = pd.merge(all_denue,naics_creative_lookup_short,
                                  left_on='codigo_act',right_on='code_int',how='inner').reset_index(drop=True)
        
        #Convert the telephone number to a string
        #NB the if else is to deal with missing values in the telefone field
        creative_denue['telefono'] = [str(int(x)) if type(x)!=float else np.nan for x in creative_denue['telefono']]
        
        
        #We will store two versions: a complete file and a short file with less variables of interest
        
        #Complete version with somewhat messy variable names
        self.denue_creative_long = creative_denue
        
        #Also store the tidy version
        #NB we need to re-concatenate it with the sector info we merged above
        self.denue_creative_short = pd.concat([tidy_denue_data(creative_denue),
                                               creative_denue[['creative_nesta_2016','creative_refined',
                                                               'creative_sector','has_creative']]],axis=1)
                                               
        
        
                                               
    def get_context_stats(self):
        '''
        This extracts contextual information that we will use to compare and normalise the creative data.
        
        NB this is only preliminary. I assume we will want to do more finely grained comparisons. We
        can modify this method to extract what we need to do that.
        
        For now we will get:
        -State counts
        -Municipio counts
        -Sizeband counts
        -Sizeband by state and municipality counts
        -Incorporation date counts
        
        '''
        
        #Load all denue data
        all_denue = self.denue_data
        
        #Extract contextual reports with a function
        self.context_stats = get_context(all_denue)
        
        
        
        

In [196]:
def tidy_denue_data(denue_data):
    '''
    This function returns a clean denue file with a few relevant variables
    We will use this for economic analysis
    NB the variables we are removing will nevertheless be important for marketing
    
    '''
    
    interesting_vars = ['id','nom_estab','raz_social','codigo_act','nombre_act','per_ocu',
                       'cod_postal','entidad','municipio','localidad','telefono','correoelec','www',
                       'tipoUniEco','latitud','longitud','fecha_alta']
    
    tidy_name_vars = ['id','name','legal_status','sector','sector_description','employee_sizeband',
                      'postcode','state','municipality','location','telephone','email','website','type_organisation',
                      'lat','lon','incorp_date']
    
    denue_selected = denue_data[interesting_vars]
    
    denue_selected.columns = tidy_name_vars
    
    return(denue_selected)

def get_context(denue_data):
    '''
    This function extracts some contextual data as part of the get_context_stats method in the ReadProcessDenue
    class.
    
    '''
    
    #Tidy data (having standard names will help processing later)
    tidy_denue = tidy_denue_data(denue_data)
    
    #Generate a bunch of counts
    state_counts, municipality_counts,sizeband_counts,incorp_date_counts = [tidy_denue[x].value_counts() for x in 
                                                         ['state','municipality',
                                                          'employee_sizeband','incorp_date']]
    
    #And a couple of crosstabs
    state_sizeb_crosstabs,municipality_sizeb_crosstabs = [pd.crosstab(tidy_denue[x],
                                                                      tidy_denue['employee_sizeband']) for x in
                                                         ['state','municipality']]
    
    #Put everything in a list
    out_list = [state_counts,municipality_counts,sizeband_counts,incorp_date_counts,
               state_sizeb_crosstabs,municipality_sizeb_crosstabs]
    
    return(out_list)
    
    
    

In [206]:
#Let's run this

#Extract a list of files in the 'sector data' directory
all_sector_files = os.listdir('/Users/jmateosgarcia/links/mexico/data/external/sector_data')
sector_files_csv = [x for x in all_sector_files if 'csv' in x]


#This is a container list where we will put the info we are interested in for now (the tidy denue data and the 
# contextual data

denue_eda_outputs = []


#We'll run it as an old school loop
for dataset in sector_files_csv:
    
    #For each dataset...
    #We print it to keep track of things
    
    print(dataset)
    
    #Create the path
    input_file_path = ext_data+'/sector_data/'+dataset
    
    #Initialise the class
    denue_object = ReadProcessDenue(input_file_path)
    
    #Read the data
    denue_object.read_file()
    
    #Get the creative data
    denue_object.get_creative()
    
    #Get the contextual data
    denue_object.get_context_stats()
    
    #Put everything in a list
    outputs_of_interest = [denue_object.denue_creative_short,denue_object.denue_creative_long,
                          denue_object.context_stats]
    
    #Append to DENUE
    denue_eda_outputs.append(outputs_of_interest) 
    
    
    



Actividad_econo%A6%FCmica.31-33_Industrias_manufactureras.csv.zip


/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (14,18,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Actividad_econo%A6%FCmica.46_Comercio_al_por_menor.Parte_3_de_4.csv.zip


/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (14,18,34) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Actividad_econo%A6%FCmica.46_Comercio_al_por_menor.Parte_4_de_4.csv.zip


...

Actividad_económica.48-49_Transportes__correos_y_almacenamiento.csv.zip


/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Actividad_económica.51_Información_en_medios_masivos.csv.zip
Actividad_económica.52_Servicios_financieros_y_de_seguros.csv.zip
Actividad_económica.53_Servicios_inmobiliarios_y_de_alquiler_de_bienes_muebles_e_intangibles.csv.zip
Actividad_económica.54_Servicios_profesionales__científicos_y_técnicos.csv.zip
Actividad_económica.55_Corporativos.csv.zip
Actividad_económica.56_Servicios_de_apoyo_a_los_negocios_y_manejo_de_desechos_y_servicios_de_remediación.csv.zip
Actividad_económica.61_Servicios_educativos.csv.zip


/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Actividad_económica.62_Servicios_de_salud_y_de_asistencia_social.csv.zip
Actividad_económica.71_Servicios_de_esparcimiento_culturales_y_deportivos__y_otros_servicios_recreativos.csv.zip
Actividad_económica.72_Servicios_de_alojamiento_temporal_y_de_preparación_de_alimentos_y_bebidas.csv.zip
Actividad_económica.81_Otros_servicios_excepto_actividades_gubernamentales.csv.zip
Actividad_económica.93_Actividades_legislativas__gubernamentales__de_impartición_de_justicia_y_de_organismos_internacionales_y_extraterritoriales.csv.zip


In [219]:
#This concatenates all the creative companies into a single dataframe
creative_tidy = pd.concat([x[0] for x in denue_eda_outputs]).reset_index(drop=True)
creative_tidy.to_csv(proc_data+'/{today}_denue_creative.csv'.format(today=today_str))

In [222]:
#And this saves all the outputs as a Python serialised (pickle) file

with open(proc_data+'/{today}_denue_processing_outputs'.format(today=today_str),'wb') as outfile:
    pickle.dump(denue_eda_outputs,outfile)


## 2. Explore data

Where we explore the DENUE data we have downloaded. 

The exploration has 2 angles:

* Descriptives
  * Sectoral distribution
  * Geography of the sector (including maps and specialisation)
  * Size
  * Incorporation dates ('age'?)
* Availability of contact details
  * This will inform the survey design and implementation 
